In [1]:
sc.list_packages()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1653926235028_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version   
-------------------------- ----------
awscli                     1.24.10   
beautifulsoup4             4.9.3     
boto                       2.49.0    
botocore                   1.26.10   
click                      7.1.2     
colorama                   0.4.4     
docutils                   0.16      
jmespath                   0.10.0    
joblib                     0.17.0    
lxml                       4.6.1     
mysqlclient                1.4.2     
nltk                       3.5       
nose                       1.3.4     
numpy                      1.16.5    
pip                        9.0.1     
py-dateutil                2.2       
pyasn1                     0.4.8     
python-dateutil            2.8.2     
python37-sagemaker-pyspark 1.4.1     
pytz                       2020.1    
PyYAML                     5.3.1     
regex                      2020.10.28
rsa                        4.7.2     
s3transfer                 0.5.2     
setuptools  

In [2]:
sc.install_pypi_package("boto3") # use boto3 to see book data in S3 bucket
sc.install_pypi_package("seaborn")
sc.install_pypi_package("pandas==0.23.2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/75/ca/d917b244919f1ebf96f7bbd5a00e4641f7e9191b0d070258f5dc10f5eaad/boto3-1.23.10-py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/10/5b/0479d7d845b5ba410ca702ffcd7f2cd95a14a4dfff1fde2637802b258b9b/seaborn-0.11.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/61/67/1a654b96309c991762ee9bc39c363fc618076b155fe52d295211cf2536c7/scipy-1.7.3.tar.gz
  Using cached https://files.pythonhosted.org/packages/df/e7/0ad4aad00d6d0314aaf97526a54a34d385f898ebb7915d112431fff452ff/matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/99/f0/f99700ef327e51d291efdf4a6de29e685c4d198cbf8531541fc84d169e0e/pandas-1.3.5.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/mnt/tmp/pip-build-5g2_5hjm/pandas/setup.py", line 650, in <mod

In [3]:
import boto3

from math import ceil
# tqdm provides a progress bar for iteration 
from tqdm.notebook import tqdm

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import (
    StructType, StructField, StringType, DateType
)
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    CountVectorizer, RegexTokenizer,
    IDF, StopWordsRemover
)
from pyspark.ml.clustering import LDA
#Spark NLP
from sparknlp.annotator import *
from sparknlp.base import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Retrieve data from S3

In [4]:
data = spark.read.csv('s3://lynette-dang-final-project-data/speech_data.csv', 
                      header=True, 
                      sep=',', 
                      escape="\"", 
                      multiLine=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 8
Total Rows: 719
root
 |-- _c0: string (nullable = true)
 |-- Unnamed: 0: string (nullable = true)
 |-- Legislator name: string (nullable = true)
 |-- Legislative Effectiveness Score: string (nullable = true)
 |-- party: string (nullable = true)
 |-- Name5: string (nullable = true)
 |-- name6: string (nullable = true)
 |-- speech: string (nullable = true)

## Proprecessing

In [6]:
data = data.withColumn('speech', f.lower(f.col('speech'))).withColumn('words',f.split('speech', ' '))
data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----------+---------------+-------------------------------+-----+--------+--------+--------------------+--------------------+
|_c0|Unnamed: 0|Legislator name|Legislative Effectiveness Score|party|   Name5|   name6|              speech|               words|
+---+----------+---------------+-------------------------------+-----+--------+--------+--------------------+--------------------+
|  0|         0|  Ralph Abraham|             0.0397605448961257|    R| abraham| abraham| mr. abraham. mr....|[, mr., abraham.,...|
|  1|         1|     Alma Adams|             1.7837893962860107|    D|   adams|   adams| ms. adams. mr. s...|[, ms., adams., m...|
|  2|         2|Robert Aderholt|             0.0198802724480628|    R|aderholt|aderholt| mr. aderholt. mr...|[, mr., aderholt....|
|  3|         3|   Pete Aguilar|             0.0497006773948669|    D| aguilar| aguilar| mr. aguilar. mad...|[, mr., aguilar.,...|
|  4|         4|     Rick Allen|             0.0397605448961257|    R|   allen|   a

## Build and Apply Spark NLP Pipeline

### 1. Document Assembler

In [11]:
from sparknlp.base import DocumentAssembler

documentAssembler = DocumentAssembler() \
     .setInputCol('speech') \
     .setOutputCol('document')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2. Tokenizer

In [12]:
from sparknlp.annotator import Tokenizer

tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('tokenized')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3.  Normalizer

In [13]:
from sparknlp.annotator import Normalizer

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4. Lemmatizer

In [14]:
from sparknlp.annotator import LemmatizerModel

lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemmatized')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]

### 5. Stopwords Remover

In [15]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[nltk_data] Downloading package stopwords to
[nltk_data]     /var/lib/livy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

In [16]:
from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('unigrams') \
     .setStopWords(eng_stopwords)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 6. N-Gram

In [17]:
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['lemmatized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
from sparknlp.annotator import PerceptronModel

pos_tagger = PerceptronModel.pretrained('pos_anc') \
    .setInputCols(['document', 'lemmatized']) \
    .setOutputCol('pos')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]

In [19]:
from sparknlp.base import Finisher

finisher = Finisher() \
     .setInputCols(['unigrams', 'ngrams', 'pos']) \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
from pyspark.ml import Pipeline

pipeline = Pipeline() \
     .setStages([documentAssembler,                  
                 tokenizer,
                 normalizer,                  
                 lemmatizer,                  
                 stopwords_cleaner, 
                 pos_tagger,
                 ngrammer,  
                 finisher])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 7. Use this pipeline to transform the speeches

In [44]:
import pyspark.sql.functions as F

speech =  data.select('speech').filter(F.col('speech').isNotNull())
processed_speech = pipeline.fit(speech).transform(speech)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
processed_speech.limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+
|              speech|   finished_unigrams|     finished_ngrams|        finished_pos|
+--------------------+--------------------+--------------------+--------------------+
| mr. abraham. mr....|[mr, abraham, mr,...|[mr, abraham, mr,...|[NNP, NN, NNP, NN...|
| ms. adams. mr. s...|[ms, adam, mr, sp...|[ms, adam, mr, sp...|[NN, NN, NNP, NN,...|
| mr. aderholt. mr...|[mr, aderholt, mr...|[mr, aderholt, mr...|[NNP, NN, NNP, NN...|
| mr. aguilar. mad...|[mr, aguilar, mad...|[mr, aguilar, mad...|[NNP, NN, NN, NN,...|
| mr. allen. mr. s...|[mr, allen, mr, s...|[mr, allen, mr, s...|[NNP, NN, NNP, NN...|
+--------------------+--------------------+--------------------+--------------------+

## Build LDA model

### 1. Count Vectorizer

In [46]:
from pyspark.ml.feature import CountVectorizer

tfizer = CountVectorizer(inputCol='finished_ngrams', outputCol='tf')
tf_model = tfizer.fit(processed_speech)
tf_result = tf_model.transform(processed_speech)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2. Tf/Idf

In [47]:
from pyspark.ml.feature import IDF

idfizer = IDF(inputCol='tf', outputCol='tf_idf')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3. Use tf/idf to build LDA model

In [48]:
from pyspark.ml.clustering import LDA

num_topics = 50
max_iter = 10

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf')
lda_model = lda.fit(tfidf_result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Get the top 50 issues and related keywords from all speeches

In [49]:
vocab = tf_model.vocabulary
from pyspark.sql import types as T


def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
num_top_words = 10

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
topics.select('topic', 'topicWords').show(50, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                                                                                                              |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[section, section_of, department, information, health, treasury, the_department, act, act_of, bankruptcy]                                                                               |
|1    |[unanimous_consent, unanimous, pennsylvania, consent, dialysis, america_small, america_small_business, organ, paycheck_protection, i_ask_unanimous]                                     |
|2    |[child_abuse, beyer, ks, dav

## Manual Issue Selection

From the topics and related words from above, I chose and assembled ten issues and related key words for the construction of issue-oriented cross-partisan social networks.

Because I only have limited amount of congressional speech (meaning that the length of each legislator's speech is not very long), I decided to only construct 10 issue-based networks instead of having more issues. The way I construct it is to record the frequency of the key words related to each issue, if any of the key words is mentioned in a legislator's speech, map this legislator to the network, and write each issue-based network to s3.

The rest of the analysis will take place locally.


In [53]:
covid = ["covid", "pandemic", "epidemic", "vaccine", "quarantine", "mask"]
bankruptcy= ["bankruptcy", "judge", "judgeship"]
child = ["child", "abuse", "children", "neglect", "hotline"]
rights = ["latino", "asian", "hate", "lgbt", "lgbtq"]
ukraine = ["putin", "ukraine", "ukranian", "ambassador", "military", "russia", "belarus", "zelensky", "war"]
business = ["loan", "small", "affordable", "application", "trade", "application"]
syria = ["military", "civil", "war", "syria"]
health_care = ["medicare", "health", "insurance", "hospital", "insulin"]
abortion = ["bible", "woman", "women", "god", "abortion"]
education = ["student", "debt", "insitution"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
from functools import partial
import re

def count_values(inp,map_list=None):  
    count = 0
    for pattern in map_list:
        if re.findall(pattern,inp):
            count += 1
    return count


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
count_values_udf = F.udf(partial(count_values,map_list=covid))
new_data = data.withColumn("count_covid", count_values_udf(F.col('speech')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
count_values_udf = F.udf(partial(count_values,map_list=bankruptcy))
new_data_2 = new_data.withColumn("count_bankruptcy", count_values_udf(F.col('speech'))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
count_values_udf = F.udf(partial(count_values,map_list=child))
new_data_3 = new_data_2.withColumn("count_child", count_values_udf(F.col('speech')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
count_values_udf = F.udf(partial(count_values,map_list=rights))
new_data_4 = new_data_3.withColumn("count_rights", count_values_udf(F.col('speech'))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
count_values_udf = F.udf(partial(count_values,map_list=ukraine))
new_data_5 = new_data_4.withColumn("count_ukraine", count_values_udf(F.col('speech'))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
count_values_udf = F.udf(partial(count_values,map_list=business))
new_data_6 = new_data_5.withColumn("count_business", count_values_udf(F.col('speech'))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
count_values_udf = F.udf(partial(count_values,map_list=syria))
new_data_7 = new_data_6.withColumn("count_syria", count_values_udf(F.col('speech'))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
count_values_udf = F.udf(partial(count_values,map_list=health_care))
new_data_8 = new_data_7.withColumn("count_health_care", count_values_udf(F.col('speech'))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
count_values_udf = F.udf(partial(count_values,map_list=abortion))
new_data_9 = new_data_8.withColumn("count_abortion", count_values_udf(F.col('speech'))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
count_values_udf = F.udf(partial(count_values,map_list=education))
final_data = new_data_9.withColumn("count_education", count_values_udf(F.col('speech'))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
final_data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----------+----------------+-------------------------------+-----+---------+---------+--------------------+--------------------+-----------+----------------+-----------+------------+-------------+--------------+-----------+-----------------+--------------+---------------+
|_c0|Unnamed: 0| Legislator name|Legislative Effectiveness Score|party|    Name5|    name6|              speech|               words|count_covid|count_bankruptcy|count_child|count_rights|count_ukraine|count_business|count_syria|count_health_care|count_abortion|count_education|
+---+----------+----------------+-------------------------------+-----+---------+---------+--------------------+--------------------+-----------+----------------+-----------+------------+-------------+--------------+-----------+-----------------+--------------+---------------+
|  0|         0|   Ralph Abraham|             0.0397605448961257|    R|  abraham|  abraham| mr. abraham. mr....|[, mr., abraham.,...|          0|               0|    

In [116]:
covid_network =final_data.select('Legislator name').where(final_data['count_covid'] > 0) 
covid_network.show()
covid_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/covid_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
| Legislator name|
+----------------+
| Robert Aderholt|
|    Pete Aguilar|
|       Andy Barr|
|Nanette Barragan|
|    Joyce Beatty|
|        Ami Bera|
|      Dan Bishop|
|      Rob Bishop|
|  Sanford Bishop|
|       Mike Bost|
|       Mo Brooks|
|    Susan Brooks|
|   Vern Buchanan|
| Michael Burgess|
|  Salud Carbajal|
|   Tony Cardenas|
|    Andre Carson|
| Matt Cartwright|
|         Ed Case|
|   James Clyburn|
+----------------+
only showing top 20 rows

In [117]:
bankruptcy_network =final_data.select('Legislator name').where(final_data['count_bankruptcy'] > 0) 
bankruptcy_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/bankruptcy_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
child_network =final_data.select('Legislator name').where(final_data['count_child'] > 0) 
child_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/child_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
rights_network =final_data.select('Legislator name').where(final_data['count_rights'] > 0) 
rights_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/rights_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
ukraine_network =final_data.select('Legislator name').where(final_data['count_ukraine'] > 0) 
ukraine_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/ukraine_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
business_network =final_data.select('Legislator name').where(final_data['count_business'] > 0) 
business_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/business_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
syria_network =final_data.select('Legislator name').where(final_data['count_syria'] > 0) 
syria_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/syria_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
health_care_network =final_data.select('Legislator name').where(final_data['count_health_care'] > 0) 
health_care_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/health_care_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
abortion_network =final_data.select('Legislator name').where(final_data['count_abortion'] > 0) 
abortion_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/abortion_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
education_network =final_data.select('Legislator name').where(final_data['count_education'] > 0) 
education_network.write.csv("s3://lynette-dang-final-project-data/issue_based_network/education_network")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Reference:
1. https://github.com/lsc4ss-s21/large-scale-personal-finance/blob/main/4_Pyspark_topic_modeling.ipynb
2. https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/2.Text_Preprocessing_with_SparkNLP_Annotators_Transformers.ipynb#scrollTo=EgdWV7yFix8e
3. https://medium.com/trustyou-engineering/topic-modelling-with-pyspark-and-spark-nlp-a99d063f1a6e
4. https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.clustering.LDA.html
